#**Srinivas**

#srinivasacademics@gmail.com


# Install all necessary libraries



In [1]:

!pip install -q streamlit pyngrok transformers torch sentence-transformers textblob scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 143.2 MB/s eta 0:00:00


# Paste NGROK Token


In [2]:
from pyngrok import ngrok

# Paste your Ngrok authentication token here
NGROK_AUTH_TOKEN = "34F6Q8gRoCiwig6coEfg5NNhS8b_4AX2tBzeukTEgwJ27gJgS"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

#Code

In [3]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool, cpu_count
from transformers import pipeline
from textblob import TextBlob
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import os

# --- Caching for Model Loading ---
@st.cache_resource # Decorator to cache the loaded model across runs
def load_sentiment_pipeline():
    """Loads and caches the Hugging Face sentiment analysis pipeline."""
    print("LOG: Caching and loading the sentiment pipeline model for the first time.")
    # Load the pre-trained model for sentiment analysis from Hugging Face
    return pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1) # Use CPU

# --- Analysis Functions ---
def run_sequential_analysis(texts, pipeline_model):
    """Performs sentiment analysis item by item using a single core."""
    print("\nLOG: --- Starting SEQUENTIAL Analysis ---")
    start_time = time.time() # Record start time
    results = [] # List to store analysis results (label, score)
    # Use st.status for better UI feedback during long operations
    with st.status("Running sequential analysis...") as status_seq:
        # Loop through each text entry
        for i, text in enumerate(texts):
            result = pipeline_model(text, truncation=True)[0] # Run model prediction
            results.append((result['label'].capitalize(), result['score'])) # Store label and score
            # Update the status message in the UI
            status_seq.update(label=f"Sequential: Processing item {i+1}/{len(texts)}...")
    processing_time = time.time() - start_time # Calculate total time
    print(f"LOG: --- Sequential analysis finished in {processing_time:.2f} seconds. ---")
    # Separate labels and scores for easier use later
    labels = [res[0] for res in results]
    scores = [res[1] for res in results]
    return labels, scores, processing_time

def process_chunk(text_chunk):
    """Worker function for parallel processing: analyzes a subset (chunk) of data."""
    pid = os.getpid() # Get the unique process ID for logging
    print(f"LOG: Parallel worker process started with ID: {pid}")
    # Each parallel worker needs to load its own instance of the pipeline
    sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1)
    # Analyze the assigned chunk of text; batch_size improves efficiency
    results = sentiment_pipeline(list(text_chunk), batch_size=8, truncation=True)
    print(f"LOG: Parallel worker {pid} finished its chunk.")
    # Return list of tuples (label, score) for this chunk
    return [(res['label'].capitalize(), res['score']) for res in results]

def run_parallel_analysis(texts):
    """Performs sentiment analysis using multiple CPU cores simultaneously."""
    print("\nLOG: --- Starting PARALLEL Analysis ---")
    num_cores = cpu_count() # Detect the number of available CPU cores
    print(f"LOG: Creating a pool of {num_cores} worker processes.")
    start_time = time.time()
    # Split the list of texts into roughly equal chunks for each core
    text_chunks = np.array_split(texts, num_cores)
    # Create a pool of worker processes
    with Pool(num_cores) as pool:
        # Distribute the chunks to the workers and collect results
        # pool.map automatically handles distributing data and gathering results
        chunk_results = pool.map(process_chunk, text_chunks)
    processing_time = time.time() - start_time
    print(f"LOG: --- Parallel analysis finished in {processing_time:.2f} seconds. ---")
    # Combine results from all chunks into a single list
    combined_results = [item for sublist in chunk_results for item in sublist]
    # Separate labels and scores
    labels = [res[0] for res in combined_results]
    scores = [res[1] for res in combined_results]
    return labels, scores, processing_time, num_cores

def get_textblob_details(text):
    """Analyzes text polarity (sentiment) and subjectivity using TextBlob."""
    sentiment = TextBlob(str(text)).sentiment # Ensure input is string, get sentiment object
    polarity = sentiment.polarity # Score from -1.0 (negative) to +1.0 (positive)
    subjectivity = sentiment.subjectivity # Score from 0.0 (objective) to 1.0 (subjective)
    # Classify sentiment label based on polarity score thresholds
    if polarity > 0.05: sentiment_label = "Positive"
    elif polarity < -0.05: sentiment_label = "Negative"
    else: sentiment_label = "Neutral" # Create a 'neutral zone' around 0
    # Return both results as a pandas Series for easy assignment to DataFrame columns
    return pd.Series([sentiment_label, subjectivity])

# --- Streamlit User Interface ---
st.set_page_config(layout="wide") # Use the full browser width for the app
st.title("Sentiment Analysis Application") # Main title displayed at the top

# --- Quick Analysis Section ---
# Allows users to test a single piece of text without uploading a file
st.header("Quick Analysis")
with st.container(border=True): # Visually group this section
    quick_text = st.text_area("Enter text for instant analysis:", height=100) # Input box
    if st.button("Analyze Single Text"): # Button to trigger analysis
        if quick_text:
            with st.spinner("Loading model and analyzing..."): # Show loading indicator
                model = load_sentiment_pipeline() # Load the LLM (cached)
                prediction = model(quick_text, truncation=True)[0] # Get prediction
                label = prediction['label'].capitalize()
                score = prediction['score']
            # Display the result using st.metric for nice formatting
            st.metric(label=f"**LLM Sentiment: {label}**", value=f"{score:.1%}")
            st.caption("Result from the Hugging Face LLM model.") # Clarify model source
        else:
            st.warning("Please enter some text to analyze.") # Handle empty input
st.markdown("<br>", unsafe_allow_html=True) # Add some vertical spacing

# --- Sidebar Configuration ---
# Sidebar is used for controls that affect the main analysis
st.sidebar.header("Batch Analysis Configuration")
# File uploader widget allows drag-and-drop or browsing
uploaded_file = st.sidebar.file_uploader("Upload Data File (.csv or .xlsx)", type=["csv", "xlsx"])

# Initialize variables in Streamlit's session state
# Session state keeps variables persistent across user interactions (button clicks, etc.)
if 'results_df' not in st.session_state: st.session_state.results_df = None # To store analysis results DataFrame
if 'performance' not in st.session_state: st.session_state.performance = None # To store timing metrics
if 'db_saved' not in st.session_state: st.session_state.db_saved = False # Flag for DB download button
if 'db_name' not in st.session_state: st.session_state.db_name = "sentiment_results.db" # Default DB filename

# --- Data Loading and Batch Analysis Setup ---
# This block only runs if a file has been uploaded
if uploaded_file:
    try:
        # Read the uploaded file into a pandas DataFrame based on its extension
        df = pd.read_csv(uploaded_file) if uploaded_file.name.endswith('.csv') else pd.read_excel(uploaded_file)
        # Dropdown menu for the user to select the column containing text
        text_column = st.sidebar.selectbox("Select Text Column:", df.columns)
        # Remove rows where the selected text column is empty before proceeding
        df_cleaned = df.dropna(subset=[text_column]).copy()

        # --- Row Selection Widgets ---
        st.sidebar.markdown("---") # Visual separator
        st.sidebar.write("**Select Number of Rows**")
        # Slider for quick, approximate selection
        slider_val = st.sidebar.slider("Drag", 10, len(df_cleaned), min(100, len(df_cleaned)), 10, label_visibility="collapsed")
        # Number input for precise entry, linked to the slider's value
        num_rows = st.sidebar.number_input("Enter number", 10, len(df_cleaned), slider_val, 10)
        st.sidebar.markdown("---")

        # Create the sample DataFrame based on the selected number of rows
        df_sample = df_cleaned.head(num_rows)
        # Extract the text data into a list for efficient processing by analysis functions
        texts = df_sample[text_column].tolist()

        # --- Analysis Execution Trigger ---
        # This code runs only when the "Run Batch Analysis" button is clicked
        if st.sidebar.button("Run Batch Analysis"):
            # Use st.status for displaying sequential steps clearly
            with st.status("Starting Analysis...", expanded=True) as status_main:
                status_main.update(label="Step 1/3: Loading sentiment model...")
                sentiment_pipeline = load_sentiment_pipeline() # Load or get the cached LLM

                status_main.update(label="Step 2/3: Running sequential analysis...")
                sequential_labels, sequential_scores, seq_time = run_sequential_analysis(texts, sentiment_pipeline)
                df_sample['LLM Sentiment'] = sequential_labels # Add results to the sample DataFrame
                df_sample['LLM Confidence'] = sequential_scores

                status_main.update(label=f"Step 3/3: Preparing parallel analysis...")
                # Use st.spinner for the parallel part as granular progress is harder to track
                with st.spinner(f"Running parallel analysis using {cpu_count()} cores..."):
                     parallel_labels, parallel_scores, par_time, num_cores = run_parallel_analysis(texts)
                # Note: We assume parallel is correct if sequential worked, primarily using it for timing.

                # Run TextBlob analysis on the text column
                df_sample[['TextBlob Sentiment', 'TextBlob Subjectivity']] = df_sample[text_column].apply(get_textblob_details)

                status_main.update(label="Analysis Complete!", state="complete", expanded=False) # Collapse status box on completion

            # Prepare DataFrames for the Execution Proof section
            seq_proof_df = pd.DataFrame({'Text': texts[:5], 'LLM Sentiment': sequential_labels[:5], 'LLM Confidence': sequential_scores[:5]})
            par_proof_df = pd.DataFrame({'Text': texts[:5], 'LLM Sentiment': parallel_labels[:5], 'LLM Confidence': parallel_scores[:5]})

            # Store results and performance metrics in session state to display them
            st.session_state.results_df = df_sample.copy()
            st.session_state.performance = {
                "sequential_time": seq_time, "parallel_time": par_time,
                "num_cores": num_cores, "text_column": text_column, # Store selected column name
                "sequential_proof": seq_proof_df, "parallel_proof": par_proof_df
            }
            st.session_state.db_saved = False # Reset DB flag as new results are generated

    except Exception as e:
        # Show error message if file loading/processing fails
        st.error(f"Error processing file: {e}")
else:
    # Message shown when the app starts, before a file is uploaded
    st.info("Upload a data file to run batch analysis.")

# --- Main Panel for Displaying Batch Results ---
# This block runs only after analysis is complete (results_df exists in session state)
if st.session_state.results_df is not None:
    results_df = st.session_state.results_df
    performance = st.session_state.performance

    # --- Section 1: Performance ---
    st.header("Batch Performance Metrics")
    with st.container(border=True): # Visual grouping
        # Display timing and speedup using st.metric
        col1, col2, col3 = st.columns(3)
        col1.metric("Sequential Time", f"{performance['sequential_time']:.2f} s")
        col2.metric(f"Parallel Time ({performance['num_cores']} Cores)", f"{performance['parallel_time']:.2f} s")
        speedup = performance['sequential_time'] / performance['parallel_time'] if performance['parallel_time'] > 0 else 0
        col3.metric("Speedup Factor", f"{speedup:.2f}x")

        # Expandable section with notes explaining performance differences
        with st.expander("Performance Notes"):
             st.markdown("""
             **Understanding Sequential vs. Parallel Speed:**
             1.  **Overhead:** Parallel processing involves starting multiple independent processes, which has a time cost.
             2.  **Model Loading:** *Each* parallel process needs to load the ~501MB sentiment model into RAM separately, consuming time and memory (Total RAM ≈ 501MB × Cores). Sequential loads it only once.
             3.  **Small Datasets:** For fewer items (< ~200), the overhead and separate model loading often make parallel *slower* than sequential.
             4.  **Large Datasets:** For more items (500+), the time saved by analyzing simultaneously outweighs the setup cost, making parallel significantly faster (often 2-3x+).
             5.  **Conclusion:** Parallelism is beneficial for large tasks but adds overhead that's noticeable on small tasks.
             """)

        # Expandable section showing first 5 results from both methods as proof
        with st.expander("Execution Proof (First 5 Results)"):
            col_proof1, col_proof2 = st.columns(2)
            # Format confidence scores as percentages for display
            proof_seq_formatted = performance['sequential_proof'].copy()
            proof_seq_formatted['LLM Confidence'] = proof_seq_formatted['LLM Confidence'].map('{:.1%}'.format)
            proof_par_formatted = performance['parallel_proof'].copy()
            proof_par_formatted['LLM Confidence'] = proof_par_formatted['LLM Confidence'].map('{:.1%}'.format)
            with col_proof1:
                st.subheader("Sequential LLM")
                st.table(proof_seq_formatted) # Use st.table for simple data display
            with col_proof2:
                st.subheader("Parallel LLM")
                st.table(proof_par_formatted)

    st.markdown("<br>", unsafe_allow_html=True) # Add vertical space

    # --- Section 2: Analysis & Comparison ---
    st.header("Batch Analysis Results")
    with st.container(border=True):
        # Checkboxes to allow users to toggle graph visibility
        st.write("**Select Visualizations to Display:**")
        show_heatmap = st.checkbox("Agreement Heatmap", value=True)
        show_distribution_bar = st.checkbox("Distribution Bar Chart", value=True)
        show_distribution_pie = st.checkbox("Distribution Pie Chart", value=False)
        show_disagreement_bar = st.checkbox("Disagreement Bar Chart", value=False)
        st.markdown("---")

        # Arrange comparison charts/info in columns
        col_left, col_right = st.columns([0.6, 0.4]) # Give more space to the left column

        with col_left: # Agreement analysis and disagreement examples
            if show_heatmap:
                st.subheader("Model Agreement Heatmap")
                # Calculate how often LLM and TextBlob results match
                comparison_matrix = pd.crosstab(results_df['TextBlob Sentiment'], results_df['LLM Sentiment'])
                # Plot using seaborn's heatmap
                fig1, ax1 = plt.subplots(figsize=(5, 3.5))
                sns.heatmap(comparison_matrix, annot=True, fmt='d', cmap='Blues', ax=ax1)
                ax1.set_ylabel('TextBlob')
                ax1.set_xlabel('LLM')
                st.pyplot(fig1, use_container_width=True) # Display the plot

            # Find rows where the sentiment predictions differ
            disagreements = results_df[results_df['TextBlob Sentiment'] != results_df['LLM Sentiment']]
            if not disagreements.empty:
                text_col_name = performance['text_column'] # Get original text column name
                # Show specific disagreement examples in an expander
                with st.expander(f"Disagreement Examples ({len(disagreements)} total)"):
                    st.caption("Subjectivity Score (Subj): 0.0=Fact, 1.0=Opinion.") # Add explanation here
                    for i, row in disagreements.head(5).iterrows():
                        st.markdown(f"**Text:** *'{row[text_col_name][:100]}...'*\n"
                                    f"- TextBlob: `{row['TextBlob Sentiment']}` (Subj: {row['TextBlob Subjectivity']:.2f})\n"
                                    f"- LLM: `{row['LLM Sentiment']}` (Conf: {row['LLM Confidence']:.1%})")
                        st.markdown("---")

            # Optional bar chart showing disagreements
            if show_disagreement_bar and not disagreements.empty:
                 st.subheader("Disagreements by LLM Sentiment")
                 # Count how many disagreements fall into each LLM sentiment category
                 disagreement_counts = disagreements['LLM Sentiment'].value_counts()
                 fig_dis, ax_dis = plt.subplots(figsize=(5, 3.5))
                 disagreement_counts.plot(kind='bar', ax=ax_dis, color=['lightcoral', 'lightskyblue', 'lightgrey'])
                 ax_dis.set_ylabel('Number of Disagreements')
                 ax_dis.set_xlabel('LLM Prediction (when TextBlob differed)')
                 ax_dis.tick_params(axis='x', rotation=0)
                 st.pyplot(fig_dis, use_container_width=True)
            elif show_disagreement_bar and disagreements.empty:
                 st.caption("No disagreements found to plot.")


        with col_right: # Sentiment distribution charts
            st.subheader("LLM Sentiment Distribution")
            # Calculate counts and percentages for each sentiment
            sentiment_counts = results_df['LLM Sentiment'].value_counts()
            total_count = len(results_df)
            perc_pos = (sentiment_counts.get("Positive", 0) / total_count) * 100
            perc_neg = (sentiment_counts.get("Negative", 0) / total_count) * 100
            perc_neu = (sentiment_counts.get("Neutral", 0) / total_count) * 100
            # Display percentages using st.metric
            met_col1, met_col2, met_col3 = st.columns(3)
            met_col1.metric("Positive", f"{perc_pos:.1f}%")
            met_col2.metric("Negative", f"{perc_neg:.1f}%")
            met_col3.metric("Neutral", f"{perc_neu:.1f}%")

            # Optional bar chart
            if show_distribution_bar:
                st.markdown("###### Distribution Bar Chart")
                fig2, ax2 = plt.subplots(figsize=(5, 3))
                sentiment_counts.plot(kind='bar', ax=ax2, color=['skyblue', 'salmon', 'lightgray'])
                ax2.set_ylabel('Count')
                ax2.tick_params(axis='x', rotation=0)
                st.pyplot(fig2, use_container_width=True)

            # Optional pie chart
            if show_distribution_pie:
                st.markdown("###### Distribution Pie Chart")
                colors = {'Positive': 'skyblue', 'Negative': 'salmon', 'Neutral': 'lightgray'}
                chart_colors = [colors.get(sentiment, '#CCCCCC') for sentiment in sentiment_counts.index]
                fig3, ax3 = plt.subplots(figsize=(5, 3))
                # Explode smallest slice slightly for better visibility
                explode = tuple([0.05 if x == sentiment_counts.min() else 0 for x in sentiment_counts])
                ax3.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90, colors=chart_colors, explode=explode)
                ax3.axis('equal') # Ensures pie chart is circular
                st.pyplot(fig3, use_container_width=True)


    st.markdown("<br>", unsafe_allow_html=True)

    # --- Section 3: Detailed Data Table ---
    st.header("Detailed Results Table")
    with st.container(border=True):
        # Explanation moved to Disagreement Expander above, keeping table clean
        # st.caption("Subjectivity Score (TextBlob): 0.0 (Objective) to 1.0 (Subjective).")
        st.caption("""
        **Subjectivity Score (TextBlob):** Measures how opinionated the text is vs. factual.
        Ranges from **0.0 (Objective - likely a fact)** to **1.0 (Subjective - likely an opinion)**.
        """)
        st.subheader("Filter Displayed Data")
        # Add filter widgets (multiselect for sentiment, checkbox for disagreements)
        col_filter1, col_filter2 = st.columns(2)
        with col_filter1:
            filter_sentiment = st.multiselect("Filter by LLM Sentiment:", results_df['LLM Sentiment'].unique(), default=results_df['LLM Sentiment'].unique())
        with col_filter2:
             show_disagreements_only = st.checkbox("Show only disagreements")

        # Apply selected filters to the results DataFrame
        filtered_df = results_df[results_df['LLM Sentiment'].isin(filter_sentiment)]
        if show_disagreements_only:
            filtered_df = filtered_df[filtered_df['LLM Sentiment'] != filtered_df['TextBlob Sentiment']]

        st.write(f"Displaying {len(filtered_df)} of {len(results_df)} results.")
        # Select and format specific columns for the main table display
        text_col_name = performance['text_column']
        cols_to_display = [text_col_name, 'LLM Sentiment', 'LLM Confidence', 'TextBlob Sentiment', 'TextBlob Subjectivity']
        cols_exist = [col for col in cols_to_display if col in filtered_df.columns]
        display_df_filtered_cols = filtered_df[cols_exist].copy()

        # Format confidence and subjectivity for display
        if 'LLM Confidence' in display_df_filtered_cols:
             display_df_filtered_cols['LLM Confidence'] = display_df_filtered_cols['LLM Confidence'].map('{:.1%}'.format)
        if 'TextBlob Subjectivity' in display_df_filtered_cols:
             display_df_filtered_cols['TextBlob Subjectivity'] = display_df_filtered_cols['TextBlob Subjectivity'].map('{:.2f}'.format)

        # Rename original text column if it's not one of the standard names
        if text_col_name not in ['LLM Sentiment', 'LLM Confidence', 'TextBlob Sentiment', 'TextBlob Subjectivity']:
             display_df_filtered_cols = display_df_filtered_cols.rename(columns={text_col_name: "Original Text"})

        # Display the final, filtered, formatted table
        st.dataframe(display_df_filtered_cols)

    st.markdown("<br>", unsafe_allow_html=True)

    # --- Section 4: Save and Export ---
    st.header("Save and Export Results")
    with st.container(border=True):
        text_col_name = performance['text_column']
        # Prepare the DataFrame for export (consistent columns)
        cols_to_export = [text_col_name, 'LLM Sentiment', 'LLM Confidence', 'TextBlob Sentiment', 'TextBlob Subjectivity']
        cols_exist_export = [col for col in cols_to_export if col in results_df.columns]
        final_export_df = results_df[cols_exist_export].rename(columns={
            text_col_name: "Original Text" # Standardize text column name for export
        })

        # Arrange export options in columns
        col_db, col_csv, col_email = st.columns(3)

        # Database Save/Download controls
        with col_db:
            st.subheader("Database")
            db_name_input = st.text_input("DB file name (.db)", value=st.session_state.db_name, key="db_name_input")
            # Update DB name if user changes it
            if db_name_input != st.session_state.db_name:
                 st.session_state.db_name = db_name_input
                 st.session_state.db_saved = False # Must re-save if name changed
            # Button to save data to SQLite
            if st.button("Save to DB"):
                try:
                    conn = sqlite3.connect(st.session_state.db_name)
                    # Write the final_export_df to the specified table
                    final_export_df.to_sql("sentiment_results", conn, if_exists="replace", index=False)
                    conn.close()
                    st.session_state.db_saved = True # Mark as saved
                    st.success(f"Saved to `{st.session_state.db_name}`.")
                except Exception as e: st.error(f"DB Error: {e}"); st.session_state.db_saved = False
            # Show download button only after successful save
            if st.session_state.db_saved:
                try:
                    # Read the saved file in binary mode
                    with open(st.session_state.db_name, "rb") as fp:
                        st.download_button(label="Download DB File", data=fp, file_name=st.session_state.db_name, mime="application/octet-stream")
                except FileNotFoundError: st.error(f"DB file not found.")
                except Exception as e: st.error(f"Error reading DB file: {e}")

        # CSV Download controls
        with col_csv:
            st.subheader("CSV File")
            csv_filename = st.text_input("CSV file name", "sentiment_analysis_results.csv")
            # Convert DataFrame to CSV bytes
            csv_data = final_export_df.to_csv(index=False).encode('utf-8')
            st.download_button("Download CSV", csv_data, csv_filename, 'text/csv')

        # Email Report controls
        with col_email:
            st.subheader("Email Report")
            recipient_email = st.text_input("Recipient Email", placeholder="Enter recipient email ID") # Email input
            if st.button("Send Email"): # Button to send
                if not recipient_email: st.warning("Enter recipient email.")
                else:
                    try:
                        # Build email message
                        msg = MIMEMultipart()
                        msg['Subject'] = "Sentiment Analysis Report"
                        msg['From'] = "websitehosting0123@gmail.com" # Sender email
                        msg['To'] = recipient_email
                        msg.attach(MIMEText("Sentiment analysis results attached.", 'plain')) # Email body
                        # Attach the CSV data
                        part = MIMEApplication(final_export_df.to_csv(index=False).encode('utf-8'), Name=csv_filename)
                        part['Content-Disposition'] = f'attachment; filename="{csv_filename}"'
                        msg.attach(part)
                        # --- Email Sending Logic ---
                        # IMPORTANT: Use a Gmail App Password here for security
                        app_password = "tzfheimxphcssuag" # Replace with your 16-digit app password
                        # Connect to Gmail's SSL SMTP server
                        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
                            server.login(msg['From'], app_password) # Login
                            server.sendmail(msg['From'], [msg['To']], msg.as_string()) # Send
                        st.success(f"Email sent to {recipient_email}!")
                    except Exception as e: st.error(f"Email Error: {e}") # Show errors

Writing app.py


#New Tunnel for getting Public URL

In [4]:
import time
from pyngrok import ngrok

# Clean up old logs
!rm -f nohup.out

# Start Streamlit app
!nohup streamlit run app.py &

# Wait for the app to start
time.sleep(10)

# Kill any existing ngrok tunnels
ngrok.kill()

# Start a new tunnel
public_url = ngrok.connect(addr=8501, proto="http")
print(f"Click the following link to view your app: {public_url}")


nohup: appending output to 'nohup.out'
Click the following link to view your app: NgrokTunnel: "https://silklike-dashingly-matt.ngrok-free.dev" -> "http://localhost:8501"


In [5]:
# Display the contents of the nohup.out log file
!cat nohup.out




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.173.96:8501

